In [1]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


## Set up parameters, create isochrone grid

In [2]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.MISTv1()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F115W', 'jwst,F182M']
filters = ['m_jwst_F115W', 'm_jwst_F182M']
iso_dir = 'isochrones/'
metallicity = 0

In [3]:
# read nbody data
path_to_data = '../../../../../../storage/jfarias/Data/Orion/massive_clusters/M3000new/sigma0p1/fiducial/sfe_ff100/00/'
snapshot = Reader.read_snapshot(path_to_data, time = 20)

# convert snapshot values from nbody units to astrophysical, extract masses
snapshot.to_physical()

# read ages for simulated stars
cluster_table = converter.to_spicea_table(snapshot)

print(cluster_table)

        mass        isMultiple        compMass            systemMass              e                 log_a               age        
------------------- ---------- --------------------- ------------------- -------------------- ------------------ ------------------
0.15548709189209567        0.0                    [] 0.15548709189209567                  nan                nan 19.927175803177988
0.05590752081313288        0.0                    [] 0.05590752081313288                  nan                nan 19.927175803177988
 15.152175805062605        0.0                    []  15.152175805062605                  nan                nan 19.896860825946256
0.19833611713212587        0.0                    [] 0.19833611713212587                  nan                nan 19.980296008051358
                ...        ...                   ...                 ...                  ...                ...                ...
 0.7087321180887948        1.0  [0.3160290611074498]  1.0247611791962445 0.0

In [4]:
masses = cluster_table['mass']
level_ages = cluster_table['age']

for i in range(len(level_ages)):
    level_ages[i] = level_ages[i] * 1e6

# convert log ages from log(Myr) to log(yr)
log_ages = np.log10(level_ages)

# print(log_ages)
print("Min log age = " + str(min(log_ages)))
print("Max log age = " + str(max(log_ages)))

# print(level_ages)
print("Min level age = " + str(min(level_ages)))
print("Max level age = " + str(max(level_ages)))

Min log age = 7.297141350870765
Max log age = 7.301155348493374
Min level age = 19821720.63142749
Max level age = 20005773.544311523


In [5]:
# find index of first binary, create array of companion masses
first_binary = 0
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        first_binary = i
        break
print(first_binary)

compMasses = []
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        compMasses.append(cluster_table['compMass'][i][0])
print(compMasses)

1352
[0.16312509029104827, 0.09636490446440792, 0.017003777999204207, 0.03636643773183382, 0.21849298240279427, 0.10166459547430746, 0.03651675631669997, 0.019132423710002903, 0.12247264926103713, 0.6385500001324462, 0.2022302569658592, 0.07471903553293849, 0.16814292142663018, 0.011007644292289143, 0.0230523225129767, 0.1609042880961581, 0.2688387764755287, 0.012694034520736917, 0.09389877662294215, 0.024808470564410257, 0.537538199511473, 0.10413768535122994, 0.1908368021249771, 0.1895631651814792, 0.09792501441534668, 0.05503713712096214, 0.014894501085541238, 0.16944725553078624, 0.03849610313773155, 0.03068110511487832, 0.010428669715904573, 0.08125256687517324, 0.16246923291724738, 0.1713331396330831, 0.010724686463596327, 0.030670575167798916, 0.02728213963323672, 0.045772770042914156, 0.15501195511789118, 0.026778635366288318, 0.016255424024083663, 0.03575262254606042, 0.1309257580510967, 0.7052331632631441, 0.05378693756462908, 0.09028542237526906, 0.03368280383489044, 0.55107

In [6]:
start = (min(level_ages) // 0.5e6) * 0.5e6
end = (max(level_ages) // 0.5e6) * 0.5e6 + 2 * 0.5e6
increment = 0.5e6  # 500,000

# Create age arrays
level_age_arr = np.arange(start, end, increment)
log_age_arr = np.log10(level_age_arr)

print(log_age_arr)
print(level_age_arr)

[7.29003461 7.30103    7.31175386]
[19500000. 20000000. 20500000.]


In [ ]:
# create isochrone grid - if this is the first time, then this is going to take an hour lmfao
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

Changing to logg=2.50 for T= 12437 logg=2.50
Changing to logg=2.50 for T= 12267 logg=2.48
Changing to logg=2.50 for T= 12099 logg=2.45
Changing to logg=2.50 for T= 11934 logg=2.43
Changing to logg=2.50 for T= 11771 logg=2.41
Changing to logg=2.00 for T=  9057 logg=1.99
Isochrone generation took 54.191121 s.
Making photometry for isochrone: log(t) = 7.29  AKs = 0.00  dist = 410
     Starting at:  2023-12-19 04:08:34.788094   Usually takes ~5 minutes
Starting filter: jwst,F115W   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.100 Msun  T =  3007 K  m_jwst_F115W = 16.28
M =   1.668 Msun  T =  7913 K  m_jwst_F115W = 10.22
M =   5.616 Msun  T = 17844 K  m_jwst_F115W = 7.48
M =  11.155 Msun  T = 20697 K  m_jwst_F115W = 4.65
M =  11.214 Msun  T = 10114 K  m_jwst_F115W = 2.44
M =  11.236 Msun  T =  3587 K  m_jwst_F115W = 0.56
M =  11.857 Msun  T =  3565 K  m_jwst_F115W = 0.47
Starting filter: jwst,F182M   Elapsed time: 14.80 seconds
Starting synthetic photometry
M =   0.100 M

In [ ]:
# identify array indeces, create array for labels and colors
idx_arr = range(0, len(log_age_arr))
if len(idx_arr) > 3:
    idx_arr = idx_arr[::2]

# Plot CMD
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

In [ ]:
import sys
sys.path.append('/scratch/wyz5rge/synthetic-hr/12-18')
import interpolatorExp as interpolator

level_ages_myr = level_ages / 1e6
# print(level_ages_myr)

# create array of stars
stars = np.empty(len(cluster_table), dtype=object)

# perform interpolation for each star
for i in range(len(stars)):
    # print(str(i) + " " + str(level_ages_myr[i]) + " " + str(masses[i]))
    stars[i] = interpolator.interpolate(level_ages_myr[i], masses[i], instances, log_age_arr, filters)
    
# convert luminosity values to solar luminosities
watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun

for i in range(len(stars)):
    if stars[i] is None:
        continue
    stars[i][0] = stars[i][0] * watts_to_lsun
    
for i in range(len(stars)):
    print(stars[i])

In [ ]:
fig, axes = py.subplots(figsize=(15, 10))

# plot CMD
py.subplot(1, 2, 1)
for i in range(len(idx_arr)):
    idx = idx_arr[i]
    if i == 0:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'lightcoral', label=level_age_arr[i])
    elif i == len(idx_arr) - 1:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[idx].points[filters[0]] - instances[idx].points[filters[1]], 
           instances[idx].points[filters[1]], 'red', label='')
    
py.xlabel('F115W - F182M')
py.ylabel('F182M')
py.gca().invert_yaxis()
py.legend()

for i in range(len(stars)):
    if stars[i] is None:
        continue
    py.plot(stars[i][3] - stars[i][4], stars[i][4], marker='o', markersize=1, color='k')
